<a href="https://colab.research.google.com/github/MWFK/TunisAir-Stock-Scrapping-Predicting/blob/main/Predict_TAIR_Stock_Value_with_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

import tensorflow as tf

In [3]:
url = 'https://raw.githubusercontent.com/MWFK/TunisAir-Stock-Scrapping-Predicting/main/TAIR_21_20_19_18_17.csv'
column_names = ['symbole', 'date', 'ouverture', 'haut', 'bas', 'cloture', 'volume']

raw_dataset = pd.read_csv(url, names=column_names,sep=';')
raw_dataset

,symbole,date,ouverture,haut,bas,cloture,volume
0,symbole,date,ouverture,haut,bas,cloture,volume
1,TAIR,02/01/2017,"0,6","0,6","0,59","0,6",4651
2,TAIR,03/01/2017,"0,6","0,6","0,59","0,6",1159
3,TAIR,04/01/2017,"0,6","0,6","0,59","0,6",19524
4,TAIR,05/01/2017,"0,6","0,6","0,59","0,6",2205
...,...,...,...,...,...,...,...
1052,TAIR,14/06/2021,"0,59","0,59","0,58","0,58",15785
1053,TAIR,15/06/2021,"0,58","0,58","0,57","0,57",28264
1054,TAIR,16/06/2021,"0,58","0,58","0,57","0,57",4176
1055,TAIR,17/06/2021,"0,57","0,57","0,56","0,57",4201
